In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:

standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
data = requests.get(standings_url)

In [4]:
#retirando apenas os links dos times da tabela
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [87]:
#tornando os links em formato completo
team_urls = [f"https://fbref.com{l}" for l in links]

In [88]:
#pegando informação dos times
team_url = team_urls[0]
data = requests.get(team_url)
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
matches.head()

C:\Users\Carlos\AppData\Local\Temp\ipykernel_11836\182362012.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]


,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes
0,2023-08-13,16:30,Premier League,Matchweek 1,Sun,Away,D,1.0,1.0,Chelsea,1.3,1.4,35.0,40096.0,Virgil van Dijk,4-3-3,Anthony Taylor,Match Report,NaN
1,2023-08-19,15:00,Premier League,Matchweek 2,Sat,Home,W,3.0,1.0,Bournemouth,3.0,1.3,64.0,53145.0,Virgil van Dijk,4-3-3,Thomas Bramall,Match Report,NaN
2,2023-08-27,16:30,Premier League,Matchweek 3,Sun,Away,W,2.0,1.0,Newcastle Utd,0.9,2.0,41.0,52214.0,Virgil van Dijk,4-3-3,John Brooks,Match Report,NaN
3,2023-09-03,14:00,Premier League,Matchweek 4,Sun,Home,W,3.0,0.0,Aston Villa,2.5,0.7,63.0,50109.0,Trent Alexander-Arnold,4-3-3,Simon Hooper,Match Report,NaN
4,2023-09-16,12:30,Premier League,Matchweek 5,Sat,Away,W,3.0,1.0,Wolves,2.5,0.6,65.0,31257.0,Andrew Robertson,4-3-3,Michael Oliver,Match Report,NaN


In [48]:
#pegando dados de chutes
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]
data = requests.get(f"https://fbref.com{links[0]}")
shooting = pd.read_html(data.text, match="Shooting")[0]
shooting.head()

C:\Users\Carlos\AppData\Local\Temp\ipykernel_11836\190749613.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


For Liverpool                                                             \
           Date   Time            Comp        Round  Day Venue Result   GF   
0    2023-08-13  16:30  Premier League  Matchweek 1  Sun  Away      D  1.0   
1    2023-08-19  15:00  Premier League  Matchweek 2  Sat  Home      W  3.0   
2    2023-08-27  16:30  Premier League  Matchweek 3  Sun  Away      W  2.0   
3    2023-09-03  14:00  Premier League  Matchweek 4  Sun  Home      W  3.0   
4    2023-09-16  12:30  Premier League  Matchweek 5  Sat  Away      W  3.0   

                       ... Standard               Expected                    \
    GA       Opponent  ...     Dist   FK PK PKatt       xG npxG npxG/Sh G-xG   
0  1.0        Chelsea  ...     17.8  0.0  0     0      1.3  1.3    0.10 -0.3   
1  1.0    Bournemouth  ...     16.8  1.0  0     1      3.0  2.1    0.09  0.0   
2  1.0  Newcastle Utd  ...     17.2  1.0  0     0      0.9  0.9    0.10  1.1   
3  0.0    Aston Villa  ...     14.7  0.0  0     0      2.5  2.5    0.15 -0.5   
4  1.0         Wolves  ...     15.8  0.0  0     0      2.5  2.5    0.16 -0.5   

          Unnamed: 25_level_0  
  np:G-xG        Match Report  
0    -0.3        Match Report  
1     0.9        Match Report  
2     1.1        Match Report  
3    -0.5        Match Report  
4    -0.5        Match Report  

[5 rows x 26 columns]

#Tratando os dados

In [49]:
shooting.columns = shooting.columns.droplevel()

In [63]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
team_data

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2023-08-13,16:30,Premier League,Matchweek 1,Sun,Away,D,1.0,1.0,Chelsea,...,4-3-3,Anthony Taylor,Match Report,NaN,13,1,17.8,0.0,0,0
1,2023-08-19,15:00,Premier League,Matchweek 2,Sat,Home,W,3.0,1.0,Bournemouth,...,4-3-3,Thomas Bramall,Match Report,NaN,25,9,16.8,1.0,0,1
2,2023-08-27,16:30,Premier League,Matchweek 3,Sun,Away,W,2.0,1.0,Newcastle Utd,...,4-3-3,John Brooks,Match Report,NaN,9,4,17.2,1.0,0,0
3,2023-09-03,14:00,Premier League,Matchweek 4,Sun,Home,W,3.0,0.0,Aston Villa,...,4-3-3,Simon Hooper,Match Report,NaN,17,4,14.7,0.0,0,0
4,2023-09-16,12:30,Premier League,Matchweek 5,Sat,Away,W,3.0,1.0,Wolves,...,4-3-3,Michael Oliver,Match Report,NaN,16,5,15.8,0.0,0,0
5,2023-09-21,18:45,Europa Lg,Group stage,Thu,Away,W,3.0,1.0,at LASK,...,4-3-3,Marco Di Bello,Match Report,NaN,13,4,16.1,0.0,1,1
6,2023-09-24,14:00,Premier League,Matchweek 6,Sun,Home,W,3.0,1.0,West Ham,...,4-3-3,Chris Kavanagh,Match Report,NaN,21,6,19.3,2.0,1,1
7,2023-09-27,19:45,EFL Cup,Third round,Wed,Home,W,3.0,1.0,Leicester City,...,4-3-3,Tim Robinson,Match Report,NaN,29,10,NaN,NaN,0,0
8,2023-09-30,17:30,Premier League,Matchweek 7,Sat,Away,L,1.0,2.0,Tottenham,...,4-3-3,Simon Hooper,Match Report,NaN,12,4,14.9,0.0,0,0
9,2023-10-05,20:00,Europa Lg,Group stage,Thu,Home,W,2.0,0.0,be Union SG,...,4-3-3,Morten Krogh,Match Report,NaN,19,9,17.2,0.0,0,0


#Pegando dados de multiplos times e multiplos anos

In [ ]:
years = list(range(2024, 2021, -1))
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

for year in years:
    print(year)
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        print(matches)
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date", how="left")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(3)


In [25]:
#Retirando informação de um único ano
year = 2024
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"


data = requests.get(standings_url)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]

links = [l.get("href") for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links] 
for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    data = requests.get(team_url)
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
    soup = BeautifulSoup(data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and 'all_comps/shooting/' in l]
    data = requests.get(f"https://fbref.com{links[0]}")
    shooting = pd.read_html(data.text, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()
    try:
        team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date", how="left")
    except ValueError:
        continue
    team_data = team_data[team_data["Comp"] == "Premier League"]
        
    team_data["Season"] = year
    team_data["Team"] = team_name
    all_matches.append(team_data)
    time.sleep(5)

C:\Users\Carlos\AppData\Local\Temp\ipykernel_3568\4027373338.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Carlos\AppData\Local\Temp\ipykernel_3568\4027373338.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\Carlos\AppData\Local\Temp\ipykernel_3568\4027373338.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Carlos\AppData\Local\Temp\ipykernel_3568\4027373338.py:22: FutureWarning: Passing literal 

In [26]:
len(all_matches)

20

In [27]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2023-08-13,16:30,Premier League,Matchweek 1,Sun,Away,D,1.0,1.0,Chelsea,...,Match Report,NaN,13.0,1.0,17.8,0.0,0.0,0.0,2024,Liverpool
1,2023-08-19,15:00,Premier League,Matchweek 2,Sat,Home,W,3.0,1.0,Bournemouth,...,Match Report,NaN,25.0,9.0,16.8,1.0,0.0,1.0,2024,Liverpool
2,2023-08-27,16:30,Premier League,Matchweek 3,Sun,Away,W,2.0,1.0,Newcastle Utd,...,Match Report,NaN,9.0,4.0,17.2,1.0,0.0,0.0,2024,Liverpool
3,2023-09-03,14:00,Premier League,Matchweek 4,Sun,Home,W,3.0,0.0,Aston Villa,...,Match Report,NaN,17.0,4.0,14.7,0.0,0.0,0.0,2024,Liverpool
4,2023-09-16,12:30,Premier League,Matchweek 5,Sat,Away,W,3.0,1.0,Wolves,...,Match Report,NaN,16.0,5.0,15.8,0.0,0.0,0.0,2024,Liverpool
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2024-04-20,NaN,Premier League,Matchweek 34,Sat,Home,NaN,NaN,NaN,Burnley,...,Head-to-Head,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024,Sheffield United
36,2024-04-27,NaN,Premier League,Matchweek 35,Sat,Away,NaN,NaN,NaN,Newcastle Utd,...,Head-to-Head,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024,Sheffield United
37,2024-05-04,NaN,Premier League,Matchweek 36,Sat,Home,NaN,NaN,NaN,Nott'ham Forest,...,Head-to-Head,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024,Sheffield United
38,2024-05-11,NaN,Premier League,Matchweek 37,Sat,Away,NaN,NaN,NaN,Everton,...,Head-to-Head,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024,Sheffield United


In [28]:
match_df.to_csv('./data/matches2024.csv', index=False)